In [9]:
# Play around CCA using sklearn
import numpy as np
import pandas as pd
import nibabel as nib
import nilearn as nil
import matplotlib.pyplot as plt
import seaborn as sns 
import csv
import sklearn
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA

In [2]:
sklearn.__version__

'0.22'

In [3]:
# get behav data from Marco
Sub = np.loadtxt('/data/backed_up/shared/mpipoly_HCPtest/batch_data/subjects.txt', dtype=str)
Task1=['WM', 'SOCIAL', 'RELATIONAL', 'GAMBLING', 'EMOTION', 'MOTOR', 'LANGUAGE','rfMRI','task']
data_dir = '/data/backed_up/shared/mpipoly_HCPtest/batch_data/'

csv2 = pd.read_csv('/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/unrestricted_mpipoly_11_19_2019_12_1_22.csv') 
df2=csv2[csv2['Subject'].isin(list(Sub))] 

newDF2=df2[['Emotion_Task_Acc','Emotion_Task_Median_RT','Language_Task_Acc','Language_Task_Median_RT','Gambling_Task_Median_RT_Larger','Gambling_Task_Median_RT_Smaller','Relational_Task_Acc','Relational_Task_Median_RT','WM_Task_Acc','WM_Task_Median_RT','Social_Task_Median_RT_TOM','Social_Task_Median_RT_Random','Social_Task_Median_RT_Unsure']].copy()

In [4]:
#get FC matrices
task_keys={}
#Glasser_Variate_X=np.array([])
#Gordon_Variate_X=np.array([])
final={}
for t in Task1:
    task_keys[t]={}
    ls_Glass=list()
    ls_Gord=list()
    final[t]={}
    for s in Sub:
        task_keys[t][s]={}
        task_keys[t][s]['Thal_X_Gord'] = (data_dir+'PYTHON_sTATS'+'/'+'{0}'+'/'+'{1}'+'/'+'{0}'+'_'+'{1}'+'_'+'Thal_X_Gord'+'.npy').format(s,t)
        #task_keys[s][t]['Thal_X_Gord'] = task_level[s]['rfMRI'].update({'Thal_X_Gord': task_level['GordParc'].get_fdata()})
        task_keys[t][s]['Thal_X_Gord'] = np.load(task_keys[t][s]['Thal_X_Gord'])
        task_keys[t][s]['Thal_X_Glass'] = (data_dir+'PYTHON_sTATS'+'/'+'{0}'+'/'+'{1}'+'/'+'{0}'+'_'+'{1}'+'_'+'Thal_X_Glass'+'.npy').format(s,t)
        task_keys[t][s]['Thal_X_Glass'] = np.load(task_keys[t][s]['Thal_X_Glass'])
        task_keys[t][s]['Thal_X_Gord'] = task_keys[t][s]['Thal_X_Gord'].flatten()
        task_keys[t][s]['Thal_X_Glass'] = task_keys[t][s]['Thal_X_Glass'].flatten()
        ls_Glass.append(np.array([task_keys[t][s]['Thal_X_Glass']]))
        ls_Gord.append(np.array(task_keys[t][s]['Thal_X_Gord']))
        #Glasser_Variate_X=np.vstack((Glasser_Variate_X,Tmp_Glass))
        #Glasser_Variate_X=np.append(Glasser_Variate_X, task_keys[t][s]['Thal_X_Glass'])
        #Gordon_Variate_X=np.append(Gordon_Variate_X, task_keys[t][s]['Thal_X_Gord'])
    final[t]={'Gordon_Variate_X': np.vstack(ls_Gord)}
    final[t].update({'Glasser_Variate_X': np.vstack(ls_Gord)})
    #print(final['WM']['Glasser_Variate_X'])
    #/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses
    np.save(f'/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses/{t}_ALL_cca_Thal_X_Gord', final[t]['Gordon_Variate_X'])
    np.save(f'/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses/{t}_ALL_cca_Thal_X_Glass', final[t]['Glasser_Variate_X'])
    
print(final['WM']['Glasser_Variate_X']) # Check Stacking
print(len(final['WM']['Glasser_Variate_X'])) # Check 100 flattened arrays are present
print(final['WM'].keys())

[[ 0.0261526   0.05646732  0.02090673 ... -0.01015351 -0.06936608
   0.1371647 ]
 [-0.00525365  0.07348273  0.01819496 ...  0.15467393 -0.05287683
   0.0529021 ]
 [ 0.08448181 -0.00367428 -0.03634613 ... -0.027576    0.04881956
   0.02245879]
 ...
 [-0.01173314  0.12081812 -0.0515199  ... -0.00310758  0.04449047
  -0.06093103]
 [-0.0188491   0.08961105 -0.00266385 ... -0.00866891 -0.04057553
   0.05887987]
 [ 0.01949048  0.00064084 -0.09660682 ... -0.08229227 -0.02524862
  -0.01831261]]
100
dict_keys(['Gordon_Variate_X', 'Glasser_Variate_X'])


In [5]:
# collect left hand size of the equation AX = BY
X = final['WM']['Glasser_Variate_X']
print(X.shape)
print(X)


(100, 844488)
[[ 0.0261526   0.05646732  0.02090673 ... -0.01015351 -0.06936608
   0.1371647 ]
 [-0.00525365  0.07348273  0.01819496 ...  0.15467393 -0.05287683
   0.0529021 ]
 [ 0.08448181 -0.00367428 -0.03634613 ... -0.027576    0.04881956
   0.02245879]
 ...
 [-0.01173314  0.12081812 -0.0515199  ... -0.00310758  0.04449047
  -0.06093103]
 [-0.0188491   0.08961105 -0.00266385 ... -0.00866891 -0.04057553
   0.05887987]
 [ 0.01949048  0.00064084 -0.09660682 ... -0.08229227 -0.02524862
  -0.01831261]]


In [6]:
# collect right hand size of the equation AX = BY
Acc=newDF2['WM_Task_Acc']
RT=newDF2['WM_Task_Median_RT']
Y = np.squeeze(np.dstack((Acc,RT)))


In [52]:
# Reduce the dimension of X, following Smith et al
pca1 = PCA(n_components = 50)
pca1.fit(X)
X_pc = pca1.transform(X)
print(np.sum(pca1.explained_variance_ratio_)) # how much varaince explained in these components?

0.5619483


In [57]:
#try CCA, and print correlation. 
# AX = U, BY = V, find A and B where corr(U, V) is maximum. Print correlation of U,V. 
#These correlations are called the Canonical correlations
cca1 = CCA(n_components=2)
cca1.fit(X_pc, Y)
X_c, Y_c = cca1.fit_transform(X_pc,Y)

print(np.corrcoef(cca1.x_scores_.T, cca1.y_scores_.T))
print(cca1.score(X_pc, Y))

[[ 1.00000000e+00  4.82682131e-17  8.70562962e-01 -2.77132572e-16]
 [ 4.82682131e-17  1.00000000e+00 -5.17802063e-04  7.58641674e-01]
 [ 8.70562962e-01 -5.17802063e-04  1.00000000e+00 -3.69458552e-16]
 [-2.77132572e-16  7.58641674e-01 -3.69458552e-16  1.00000000e+00]]
0.6474458811159715


/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [62]:
# do a permutation test

n_perm = 100
null_scores = np.zeros(n_perm)

for i in np.arange(n_perm):
    
    cca_r = CCA(n_components=2)
    x_rand = np.random.permutation(X_pc)
    cca_r.fit(x_rand, Y)
    X_c, Y_c = cca_r.fit_transform(x_rand,Y)
    
    
    null_scores[i] = cca_r.score(x_rand, Y)
    


/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform

/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform

/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform

/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform

0.4044746400599471
0.14299091440936315


/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/kahwang/.local/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform

In [66]:
print(np.mean(null_scores))
print(np.percentile(null_scores, 5))
print(np.percentile(null_scores, 95))

0.4044746400599471
0.20328106108004979
0.6355103068352936


In [217]:
# weights are the A and B in AX = BY # loadings and rotations are the PCA loadings which we will not use
print(cca1.x_loadings_.shape)
print(cca1.x_weights_.shape)
print(cca1.x_rotations_.shape)

(844488, 2)
(844488, 2)
(844488, 2)


In [67]:
# this is the steve smith approach where we correlate scores with original FC
r_vals = np.zeros(844488)

for i in np.arange(844488):
    r_vals[i]=np.corrcoef(cca1.x_scores_[:,0], X[:,i])[0,1]
    
    

In [68]:
print(np.mean(r_vals))
print(np.std(r_vals))
print(np.max(r_vals))


-0.0053512413992918145
0.10474848101428648
0.47332983865832645
